In [1]:
# Import
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F # relu, tanh
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [2]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x 

In [64]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [3]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Hyperparameters
input_size = 784
num_class = 10
learning_rate = 0.001
batch_size = 256 
num_epochs = 3

In [5]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                          shuffle=True)

In [6]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_class).to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train Network
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]:", end='')
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()





Epoch [1/3]:

100%|██████████| 235/235 [00:04<00:00, 50.90it/s]


Epoch [2/3]:

100%|██████████| 235/235 [00:04<00:00, 52.22it/s]


Epoch [3/3]:

100%|██████████| 235/235 [00:04<00:00, 51.96it/s]


In [8]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # model.eval()

    with torch.no_grad():
        if loader.dataset.train:
            print("Checking accuracy on training data")
        else:
            print("Checking accuracy on test data")
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x) # shape: 64x10
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        
        acc = round(float(num_correct)/float(num_samples)*100, 2)
        print(f'Got {num_correct} / {num_samples} with accuracy \
              {acc}')
    
    # model.train()
    # return acc
            

In [12]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on test data
Got 3609 / 10000 with accuracy               36.09
